In [1]:
import glob
from qcfractal.interface.models import ResultRecord

In [14]:
output_json = glob.glob("output/*.json")

In [15]:
ResultRecord.parse_file(output_json[0])

ResultRecord(id='1', status='COMPLETE')

In [16]:
for file in output_json:
    try:
        record = ResultRecord.parse_file(file)
#         print(file)
    except:
        print(file)
        pass
    else:
        if record.status == "ERROR":
            print("error, ", file)

INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 12.10ms
INFO:tornado.application:Acquired 0 new tasks.
INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 12.77ms
INFO:tornado.application:Acquired 0 new tasks.
INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 13.85ms
INFO:tornado.application:Acquired 0 new tasks.
INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 12.81ms
INFO:tornado.application:Acquired 0 new tasks.


In [7]:
print(output_json)

['output/ethanol_c2_o1_resp2_gas_energy.json', 'output/nme2ala2_c2_o1_resp2_water_energy.json', 'output/nme2ala2_c1_o2_resp2_gas_energy.json', 'output/nme2ala2_c2_o4_gas_energy.json', 'output/nme2ala2_c2_o1_resp2_gas_energy.json', 'output/nme2ala2_c1_o3_gas_energy.json', 'output/methylammonium_c1_o2_resp2_gas_energy.json', 'output/nme2ala2_c1_o4_gas_energy.json', 'output/nme2ala2_c2_o3_gas_energy.json', 'output/nme2ala2_c2_o3_resp2_gas_energy.json', 'output/nme2ala2_c1_o4_resp2_water_energy.json', 'output/nme2ala2_c2_o2_gas_energy.json', 'output/ethanol_c2_o1_gas_energy.json', 'output/methylammonium_c1_o1_gas_energy.json', 'output/ethanol_c1_o1_resp2_water_energy.json', 'output/nme2ala2_c1_o3_resp2_water_energy.json', 'output/nme2ala2_c1_o4_resp2_gas_energy.json', 'output/ethanol_c1_o1_gas_energy.json', 'output/nme2ala2_c1_o2_gas_energy.json', 'output/nme2ala2_c2_o2_resp2_water_energy.json', 'output/nme2ala2_c1_o3_resp2_gas_energy.json', 'output/nme2ala2_c2_o4_resp2_water_energy.json',

In [1]:
#!/usr/bin/env python


import pathlib
import time
from typing import Optional
import tempfile
import pathlib
import glob
import shutil
import subprocess
import shutil
import os

from qcfractal.postgres_harness import find_port, FractalConfig, atexit
# from qcfractal.postgres_harness import PostgresHarness as QCPostgresHarness
# from qcfractal.testing import TemporaryPostgres as QCTemporaryPostgres
from psiresp.testing import PostgresHarness, TemporaryPostgres


from qcfractal.interface.models import ResultRecord
from qcfractal.storage_sockets import storage_socket_factory
from qcfractal import FractalSnowflake, FractalSnowflakeHandler
import qcfractal.interface as ptl
import qcelemental as qcel


POSTGRES_SERVER_BACKUP = "database.psql"






In [2]:
storage = TemporaryPostgres(database_name="test_psiresp")
storage.psql.restore_database(POSTGRES_SERVER_BACKUP)

output_json = glob.glob("output/*.json")
print(storage)
records = []
i = 500
for file in output_json:
    try:
        record = ResultRecord.parse_file(file)
        
    except:
        print(file)
    else:
        dct = record.dict()
        KEEP_KEYS = "version driver program molecule method basis id protocols extras status provenance keywords"
        dct = {k: v for k, v in dct.items() if k in KEEP_KEYS}
        record = ResultRecord(**dct)
        records.append(record)
        i += 1

print(len(records))



20


In [3]:
records[0].dict()

{'id': '1',
 'hash_index': None,
 'procedure': 'single',
 'program': 'psi4',
 'version': 1,
 'protocols': {'wavefunction': <WavefunctionProtocolEnum.orbitals_and_eigenvalues: 'orbitals_and_eigenvalues'>},
 'extras': {'_qcfractal_tags': {'program': 'psi4', 'keywords': '1'},
  'qcvars': {'CURRENT DIPOLE X': 0.6999663317016805,
   'CURRENT DIPOLE Y': -0.4683628892897536,
   'CURRENT DIPOLE Z': 1.8820251791438678,
   'CURRENT ENERGY': -154.0382183256945,
   'CURRENT REFERENCE ENERGY': -154.0382183256945,
   'DFT FUNCTIONAL TOTAL ENERGY': -154.0382183256945,
   'DFT TOTAL ENERGY': -154.0382183256945,
   'DFT VV10 ENERGY': 0.0,
   'DFT XC ENERGY': -15.031655542876312,
   'GRID ELECTRONS ALPHA': 13.000000719021939,
   'GRID ELECTRONS BETA': 13.000000719021939,
   'GRID ELECTRONS TOTAL': 26.000001438043878,
   'NUCLEAR REPULSION ENERGY': 43.43159637276381,
   'ONE-ELECTRON ENERGY': -294.70309970532696,
   'PCM POLARIZATION ENERGY': 0.0,
   'PE ENERGY': 0.0,
   'SCF DIPOLE X': 0.699966331701680

In [4]:
tmp_server = FractalSnowflake(
    max_workers=1,
    storage_project_name="test_psiresp",
    storage_uri=storage.psql.database_uri(),
    reset_database=False,
    start_server=True,
)

INFO:tornado.application:FractalServer:
INFO:tornado.application:    Name:          QCFractal Snowflake Instance
INFO:tornado.application:    Version:       v0.15.6
INFO:tornado.application:    Address:       http://localhost:58577/
INFO:tornado.application:    Database URI:  postgresql://localhost:58564/
INFO:tornado.application:    Database Name: test_psiresp
INFO:tornado.application:    Query Limit:   1000000

INFO:tornado.application:Waiting on queue_manager to build.
INFO:tornado.application:FractalServer successfully started.



In [5]:

socket = storage_socket_factory(tmp_server._storage_uri,
                                project_name="test_psiresp",
                                skip_version_check=True,
                                )


INFO:tornado.application:GET: Information
INFO:tornado.access:200 GET /information (::1) 1.25ms
INFO:tornado.application:QueueManager:
INFO:tornado.application:    Version:         v0.15.6

INFO:tornado.application:    Queue Adapter:
INFO:tornado.application:        <ExecutorAdapter client=<ProcessPoolExecutor max_workers=1>>

INFO:tornado.application:QueueManager: New active manager FractalServer-vcv074104.vpn.uci.edu-aa8b9f13-09c7-476c-80ab-c9b2a99ef9b5 detected.
INFO:tornado.access:200 PUT /queue_manager (::1) 13.95ms
INFO:tornado.application:QueueManager successfully started.

INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 12.64ms
INFO:tornado.application:Acquired 0 new tasks.
INFO:tornado.access:200 PUT /queue_manager (::1) 9.15ms
INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Se

{'id': '1',
 'hash_index': None,
 'procedure': 'single',
 'program': 'psi4',
 'version': 1,
 'protocols': {'wavefunction': <WavefunctionProtocolEnum.orbitals_and_eigenvalues: 'orbitals_and_eigenvalues'>},
 'extras': {'_qcfractal_tags': {'program': 'psi4', 'keywords': '1'},
  'qcvars': {'CURRENT DIPOLE X': 0.6999663317016805,
   'CURRENT DIPOLE Y': -0.4683628892897536,
   'CURRENT DIPOLE Z': 1.8820251791438678,
   'CURRENT ENERGY': -154.0382183256945,
   'CURRENT REFERENCE ENERGY': -154.0382183256945,
   'DFT FUNCTIONAL TOTAL ENERGY': -154.0382183256945,
   'DFT TOTAL ENERGY': -154.0382183256945,
   'DFT VV10 ENERGY': 0.0,
   'DFT XC ENERGY': -15.031655542876312,
   'GRID ELECTRONS ALPHA': 13.000000719021939,
   'GRID ELECTRONS BETA': 13.000000719021939,
   'GRID ELECTRONS TOTAL': 26.000001438043878,
   'NUCLEAR REPULSION ENERGY': 43.43159637276381,
   'ONE-ELECTRON ENERGY': -294.70309970532696,
   'PCM POLARIZATION ENERGY': 0.0,
   'PE ENERGY': 0.0,
   'SCF DIPOLE X': 0.699966331701680

INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 12.42ms
INFO:tornado.application:Acquired 0 new tasks.
INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 15.27ms
INFO:tornado.application:Acquired 0 new tasks.
INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 14.26ms
INFO:tornado.application:Acquired 0 new tasks.
INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 13.46ms
INFO:tornado.application:Acquired 0 new tasks.
INFO:tornado.application:Task statistics unavailable

In [6]:
records[0].stdout

'500'

In [7]:
dir(records[0])

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__base_doc__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__model_autodoc_applied__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__validators__',
 '_abc_impl',
 '_calculate_keys',
 '_decompose_class',
 '_enforce_dict_if_root',
 '_get_value',
 '_hash_indices',
 '_init_private_attrib

INFO:tornado.application:GET: Information
INFO:tornado.access:200 GET /information (::1) 2.26ms
INFO:tornado.application:QueueManager:
INFO:tornado.application:    Version:         v0.15.6

INFO:tornado.application:    Queue Adapter:
INFO:tornado.application:        <ExecutorAdapter client=<ProcessPoolExecutor max_workers=1>>

INFO:tornado.application:QueueManager: New active manager FractalServer-vcv074104.vpn.uci.edu-01c44f21-4e04-4e48-92b6-8b96216eced1 detected.
INFO:tornado.access:200 PUT /queue_manager (::1) 15.71ms
INFO:tornado.application:QueueManager successfully started.

INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 15.42ms
INFO:tornado.application:Acquired 0 new tasks.
INFO:tornado.access:200 PUT /queue_manager (::1) 9.27ms


In [6]:
socket.add_results(records)
print(socket)

storage.psql.backup_database(POSTGRES_SERVER_BACKUP)
print(f"Saved to {POSTGRES_SERVER_BACKUP}")

<SQLAlchemySocket: address='postgresql+psycopg2://localhost:58564/test_psiresp`>


INFO:tornado.application:Task statistics unavailable until first tasks return


Saved to database.psql


INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 11.62ms
INFO:tornado.application:Acquired 0 new tasks.
INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 15.67ms
INFO:tornado.application:Acquired 0 new tasks.
INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 13.06ms
INFO:tornado.application:Acquired 0 new tasks.
INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 tasks.
INFO:tornado.access:200 GET /queue_manager (::1) 15.40ms
INFO:tornado.application:Acquired 0 new tasks.
INFO:tornado.application:Task statistics unavailable until first tasks return
INFO:tornado.application:QueueManager: Served 0 task